In [279]:
import pandas as pd
import numpy as np

# Load

In [280]:
data = pd.read_csv("../data/ifsc_data_raw.csv")

C:\Users\Saul\AppData\Local\Temp\ipykernel_7116\1076732697.py:1: DtypeWarning: Columns (4,6,8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/ifsc_data_raw.csv")


In [281]:
census = pd.read_excel("../data/census.xls")

# EDA

In [282]:
display(data.info())
display(data.sample())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113077 entries, 0 to 113076
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   entry_discipline                   113077 non-null  object 
 1   entry_category                     113077 non-null  object 
 2   entry_rank                         112773 non-null  float64
 3   entry_qualification_rank           78801 non-null   float64
 4   entry_qualification_score          78985 non-null   object 
 5   entry_semi_final_rank              15625 non-null   float64
 6   entry_semi_final_score             15629 non-null   object 
 7   entry_final_rank                   21406 non-null   float64
 8   entry_final_score                  21361 non-null   object 
 9   athlete_id                         113077 non-null  int64  
 10  athlete_firstname                  113077 non-null  object 
 11  athlete_lastname                   1130

None

,entry_discipline,entry_category,entry_rank,entry_qualification_rank,entry_qualification_score,entry_semi_final_rank,entry_semi_final_score,entry_final_rank,entry_final_score,athlete_id,...,athlete_paraclimbing_sport_class,athlete_height,athlete_speed_personal_best_score,athlete_speed_personal_best_date,athlete_speed_personal_best_round,event_id,event_name,event_start_date,event_end_date,event_location
44187,lead,women,42.0,42.0,27+ 42. | 28 43. [42.99],NaN,NaN,NaN,NaN,653,...,NaN,NaN,13.07,2019-05-05,Qualification,949,"IFSC Climbing Worldcup (L, S) - Arco (ITA) 2016",2016-08-26,2016-08-27,ArcoWC


In [283]:
# data[data["entry_category"].isin(["women", "men"])].count()
display(data.groupby("entry_category").size())

entry_category
juniors female     5256
juniors male       8145
men               37411
women             26686
youth a female     7887
youth a male      10058
youth b female     8507
youth b male       9127
dtype: int64

In [284]:
data["entry_discipline"].unique()
# data[data["entry_discipline"].isin(["boulder", "lead"])].count()
display(data.groupby("entry_discipline").size())

entry_discipline
boulder     39653
combined     1895
lead        51804
speed       19725
dtype: int64

In [285]:
display(np.sort(data["athlete_country"].unique()))

array(['AND', 'ARG', 'AUS', 'AUT', 'AZE', 'BEL', 'BIH', 'BLR', 'BOL',
       'BOT', 'BRA', 'BRU', 'BUL', 'CAM', 'CAN', 'CFR', 'CHI', 'CHN',
       'COL', 'CRC', 'CRO', 'CYP', 'CZE', 'DEN', 'ECU', 'ESA', 'ESP',
       'EST', 'FIN', 'FRA', 'GBR', 'GEO', 'GER', 'GRE', 'GUA', 'GUM',
       'HKG', 'HON', 'HUN', 'INA', 'IND', 'IRI', 'IRL', 'IRQ', 'ISL',
       'ISR', 'ITA', 'JOR', 'JPN', 'KAZ', 'KGZ', 'KOR', 'KSA', 'KUW',
       'LAT', 'LBN', 'LTU', 'LUX', 'MAC', 'MAS', 'MEX', 'MGL', 'MKD',
       'MRI', 'NED', 'NEP', 'NGR', 'NOR', 'NZL', 'New Caledonia', 'PAK',
       'PER', 'PHI', 'POL', 'POR', 'PUR', 'ROU', 'RSA', 'RUS', 'SGP',
       'SLO', 'SRB', 'SRI', 'SUI', 'SVK', 'SWE', 'THA', 'TPE', 'TUR',
       'UGA', 'UKR', 'USA', 'UZB', 'VEN'], dtype=object)

In [286]:
# display(np.sort(data["event_name"].unique()))

In [287]:
data["entry_discipline"].unique()

array(['boulder', 'lead', 'speed', 'combined'], dtype=object)

In [288]:
display(census.info())
display(census.sample())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 69 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            264 non-null    float64
 5   1961            264 non-null    float64
 6   1962            264 non-null    float64
 7   1963            264 non-null    float64
 8   1964            264 non-null    float64
 9   1965            264 non-null    float64
 10  1966            264 non-null    float64
 11  1967            264 non-null    float64
 12  1968            264 non-null    float64
 13  1969            264 non-null    float64
 14  1970            264 non-null    float64
 15  1971            264 non-null    float64
 16  1972            264 non-null    float64
 17  1973            264 non-null    flo

None

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
201,Romania,ROU,"Population, total",SP.POP.TOTL,18406905.0,18555250.0,18676550.0,18797850.0,18919126.0,19031576.0,...,19815616.0,19702267.0,19588715.0,19473970.0,19371648.0,19265250.0,19122059.0,19048502.0,19059479.0,19069340.0


# Clean

In [289]:
# Only adults
data = data[data["entry_category"].isin(["women", "men"])]
display(data.groupby("entry_category").size())

# Only boulder or lead
data = data[data["entry_discipline"].isin(["boulder", "lead", "speed"])]
display(data.groupby("entry_discipline").size())

# Only top 10
data = data[data["entry_rank"] <= 10]

# Event date
data["event_start_date"] = pd.to_datetime(data["event_start_date"])

data["event_year"] = data["event_start_date"].dt.year
data["event_month"] = data["event_start_date"].dt.month
data["event_day"] = data["event_start_date"].dt.day

# Only last decade
data = data[data["event_year"] >= 2014]
display(data.groupby("event_year").size())

# Create age column and drop athlete birthday
data = data.dropna(subset=["athlete_birthday"])
data["athlete_birthday"] = pd.to_datetime(data["athlete_birthday"], errors="coerce")

before_birthday = (
    (data["event_month"] < data["athlete_birthday"].dt.month)
    | (
        (data["event_month"] == data["athlete_birthday"].dt.month) &
        (data["event_day"] < data["athlete_birthday"].dt.day)
    )
)

data["athlete_age_at_event"] = (
    data["event_year"]
    - data["athlete_birthday"].dt.year
    - before_birthday
)
data = data[data["athlete_age_at_event"] >= 17]


display(data.groupby("athlete_age_at_event").size())

# Location
data["event_location"] = data["event_location"].str.replace("WC$|WCH |ACH ", "", regex=True).str.strip()
location_mapping = {
    "CAMPECH17": "Campitello di Fassa",
    "Ibarra, ECU": "Ibarra",
    "Neom Beach Games Mountain Village": "Bajdah",
    "BG19": "Doha",
    "XiamenWC": "Xiamen",
    "PolandECH": "Zakopane",
    "ACH Palembang": "Palembang",
    "Briancon": "Briançon",
    "EdinCH19": "Edinburgh",
    "Inns19": "Innsbruck",
    "InnsbruckECH15": "Innsbruck",
    "China_Open_17": "Guangzhou",
    "China_Open_18": "Guangzhou",
    "China_Open_Q": "Guangzhou",
    "PAC Guayaquil": "Guayaquil",
    "UCH16": "Shanghai",
    "UCH18": "Bratislava",
    "Asia_Cup_17": "Wanxianshan",
    "WG_2017": "Wroclaw",
    "MunichECH": "Munich",
    "Oceania Championship": "Sydney",
    "EUC15": "Katowice",
    "ChamECH15": "Chamonix",
    "Toronto15": "Toronto",
    "Promo Sheffield": "Sheffield",
    "Arco_Open": "Arco",
    "Arco Rockmaster": "Arco",
    "NaviMumbai": "Navi Mumbai",
    "Salt Lake CIty": "Salt Lake City",
    "Taian": "Tai'an"
}

data["event_location"] = data["event_location"].replace(location_mapping)

data.loc[(data["event_location"] == "Asia_Cup_18") & (data["event_month"] == 3), "event_location"] = "Hong Kong"
data.loc[(data["event_location"] == "Asia_Cup_18") & (data["event_month"] == 6), "event_location"] = "Bangkok"

data = data.dropna(subset=["event_location"])

display(np.sort(data["event_location"].unique()))

# Athlete countries in ISO-3
iso3_mapping = {
    "GER": "DEU",
    "GRE": "GRC",
    "DEN": "DNK",
    "PHI": "PHL",
    "POR": "PRT",
    "NED": "NLD",
    "MRI": "MUS",
    "RSA": "ZAF",
    "SUI": "CHE",
    "TPE": "TWN",
    "ESA": "SLV",
    "PUR": "PRI",
    "CAM": "KHM",
    "CHI": "CHL",
    "IRI": "IRN",
    "INA": "IDN",
    "KUW": "KWT",
    "MAS": "MYS",
    "BRU": "BRN",
    "CRO": "HRV",
    "SLO": "SVN",
    "HON": "HND",
    "CRC": "CRI",
    "GUA": "GTM",
    "BOT": "BWA",
    "MGL": "MNG",
    "New Caledonia": "NCL",
    "CFR": "RUS",
    "LAT": "LVA",
    "BUL": "BGR"
}

data["athlete_country"] = data["athlete_country"].replace(iso3_mapping)

# Ranks to int
data["entry_rank"] = data["entry_rank"].astype("int32")

# Max rank of each athlete
data["athlete_max_rank"] = (
    data.groupby("athlete_id")["entry_rank"]
      .transform("min")
)

# Best ranks
data["athlete_country_total_gold"] = data.groupby("athlete_country")["entry_rank"].transform(lambda x: (x == 1).sum())
data["athlete_country_total_silver"] = data.groupby("athlete_country")["entry_rank"].transform(lambda x: (x == 2).sum())
data["athlete_country_total_bronze"] = data.groupby("athlete_country")["entry_rank"].transform(lambda x: (x == 3).sum())

display(np.sort(data["athlete_country_total_gold"].unique()))
display(np.sort(data["athlete_country_total_silver"].unique()))
display(np.sort(data["athlete_country_total_bronze"].unique()))

# NaN speeds to dummy
data["athlete_speed_personal_best_score"] = data["athlete_speed_personal_best_score"].replace(np.nan, 999.99)

# Country populations
census = census[["Country Code", "2024"]]
census.loc[len(census)] = ["TWN", 23923272] # Añado Taiwan a mano

data = pd.merge(
    data,
    census,
    how="left",
    left_on="athlete_country",
    right_on="Country Code"
).drop(columns=["Country Code"])

data = data.rename(columns={"2024": "athlete_country_population"})
data["athlete_country_population"] = data["athlete_country_population"].astype("int64")

entry_category
men      37411
women    26686
dtype: int64

entry_discipline
boulder    25634
lead       25964
speed      11706
dtype: int64

event_year
2014    639
2015    470
2016    552
2017    667
2018    612
2019    442
2020     53
2021    409
2022    800
2023    699
2024    891
dtype: int64

athlete_age_at_event
17    315
18    454
19    518
20    657
21    636
22    563
23    508
24    483
25    420
26    346
27    295
28    252
29    185
30    112
31     61
32     45
33     29
34     33
35     13
36     11
37     11
38      3
39      1
40      2
dtype: int64

array(['Arco', 'Augsburg', 'Bajdah', 'Baku', 'Bangkok', 'Bern',
       'Birmingham', 'Bologna', 'Bratislava', 'Briançon', 'Brixen',
       'Budapest', 'Campitello di Fassa', 'Chambéry', 'Chamonix',
       'Chongqing', 'Curitiba', 'Doha', 'Duyun', 'Edinburgh', 'Gaflenz',
       'Genova', 'Gijon', 'Grindelwald', 'Guangzhou', 'Guayaquil',
       'Hachioji', 'Haiyang', 'Hamburg', 'Hangzhou', 'Hong Kong',
       'Ibarra', 'Imst', 'Indonesia', 'Innsbruck', 'Inzai', 'Jakarta',
       'Johannesburg', 'Katowice', 'Kazo', 'Keqiao', 'Klagenfurt',
       'Koper', 'Krakow', 'Kranj', 'Kurayoshi', 'Laval', 'Liébana',
       'Ljubljana', 'Lombok', 'Loverval', 'Lublin', 'Madrid, ESP',
       'Manila', 'Mar del Plata', 'Meiringen', 'Melbourne',
       'Mezzolombardo', 'Mokpo', 'Moscow', 'Mumbai', 'Munich', 'Nanjing',
       'Navi Mumbai', 'Neom', 'Ningbo', 'Palembang', 'Paris', 'Perth',
       'Prague', 'Pretoria', 'Puurs', 'Rome', 'Saanich15',
       'Salt Lake City', 'Santiago', 'Seoul', 'Shanghai'

array([ 0,  1,  2,  4,  5,  7,  8, 11, 13, 14, 16, 17, 19, 24, 25, 30, 31,
       35, 39, 52, 63, 67, 79])

array([ 0,  1,  2,  3,  5,  6,  7,  8, 13, 15, 17, 21, 22, 25, 26, 29, 35,
       40, 60, 64, 96])

array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 14, 15, 20, 21, 22, 23, 25,
       27, 28, 33, 39, 59, 66, 77])

In [ ]:
# Drop unused columns
cols_to_drop = [
    "entry_qualification_rank",
    "entry_qualification_score",
    "entry_semi_final_rank",
    "entry_semi_final_score",
    "entry_final_rank",
    "entry_final_score",
    "athlete_id",
    "athlete_paraclimbing_sport_class",
    "athlete_height",
    # "athlete_speed_personal_best_score",
    "athlete_speed_personal_best_date",
    "athlete_speed_personal_best_round",
    "event_id",
    "event_end_date",
    "athlete_gender",
    "athlete_birthday",
    "event_start_date"
]
try:
    data = data.drop(columns=cols_to_drop)
except:
    pass

data = data.reset_index(drop=True)

In [291]:
display(data.info())
display(data.sample())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5919 entries, 0 to 5918
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   entry_discipline                   5919 non-null   object 
 1   entry_category                     5919 non-null   object 
 2   entry_rank                         5919 non-null   int32  
 3   athlete_firstname                  5919 non-null   object 
 4   athlete_lastname                   5919 non-null   object 
 5   athlete_country                    5919 non-null   object 
 6   athlete_speed_personal_best_score  5919 non-null   float64
 7   event_name                         5919 non-null   object 
 8   event_location                     5919 non-null   object 
 9   event_year                         5919 non-null   int32  
 10  event_month                        5919 non-null   int32  
 11  event_day                          5919 non-null   int32

None

,entry_discipline,entry_category,entry_rank,athlete_firstname,athlete_lastname,athlete_country,athlete_speed_personal_best_score,event_name,event_location,event_year,event_month,event_day,athlete_age_at_event,athlete_max_rank,athlete_country_total_gold,athlete_country_total_silver,athlete_country_total_bronze,athlete_country_population
5779,speed,men,8,Libor,HROZA,CZE,5.61,"IFSC Climbing World Championship (L, S) - Gijo...",Gijon,2014,9,8,27,1,19,22,9,10882164


# Export

In [292]:
data.to_csv("../data/ifsc_data_clean.csv")